In [2]:
import pandas as pd

In [3]:
data = pd.read_csv(r'C:\Users\kyungmyung\Desktop\notebook\online.csv')

In [4]:
data

,store_num,product_num,sales,likes,discount,shippirce
0,0,1,NaN,39000.0,0.10,NaN
1,1,1,NaN,10000.0,0.52,NaN
2,2,1,NaN,82000.0,NaN,NaN
3,3,1,NaN,36000.0,0.41,NaN
4,1,2,NaN,10000.0,0.71,NaN
...,...,...,...,...,...,...
15503,969,114,0.0,11000.0,NaN,0.0
15504,969,115,0.0,11000.0,NaN,0.0
15505,969,116,0.0,11000.0,NaN,0.0
15506,969,117,0.0,11000.0,NaN,0.0


In [5]:
import numpy as np

In [6]:
data = data.fillna(0)

In [7]:
y = data['sales']
x = data['discount'] * 100

In [8]:
variance_x = np.var(x)  
print(variance_x)

549.3780394797467


In [9]:
covariance_xy = np.cov(x,y)[0,1]
print(covariance_xy)

-2411096.5809810506


In [10]:
beta1 = (covariance_xy/variance_x)
print(beta1)

-4388.77495588343


In [11]:
x_mean = np.mean(x)
print(x_mean)

26.747291720402373


In [12]:
y_mean = np.mean(y)
print(y_mean)

518421.77921072993


In [13]:
beta1 = np.sum((x - x_mean) * (y - y_mean)) / np.sum((x - x_mean)**2)
print(beta1)

-4388.491955176785


In [14]:
beta0 = y_mean - beta1*x_mean

In [15]:
residuals = y - (beta0 + beta1*x)

In [16]:
n = len(x)

In [17]:
s2 = np.sum(residuals**2) / (n - 2)

In [18]:
SE_beta1 = np.sqrt(s2 / np.sum((x - x_mean)**2))
print(SE_beta1)

1804.335783909066


In [19]:
beta1t = beta1/SE_beta1
print(beta1t)

-2.4321924967143222


In [20]:
from scipy import stats

In [21]:
p_beta1 = 2 * (1 - stats.t.cdf(np.abs(beta1t), df=n-2))
print(p_beta1)

0.015018967581190257


In [22]:
import numpy as np
import statsmodels.api as sm

In [23]:
x = sm.add_constant(x)

In [24]:
model = sm.OLS(y, x).fit()

In [25]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     5.916
Date:                Wed, 03 Dec 2025   Prob (F-statistic):             0.0150
Time:                        17:24:43   Log-Likelihood:            -2.6202e+05
No. Observations:               15508   AIC:                         5.240e+05
Df Residuals:                   15506   BIC:                         5.241e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.358e+05   6.42e+04      9.908      0.0

In [26]:
sales_dummy = []
for i in range(len(data)):
    if data['sales'][i]==0:
        sales_dummy.append(0)
    else:
        sales_dummy.append(1)


In [27]:
y = sales_dummy

In [28]:
x = data[['discount','likes','shippirce']]

In [29]:
x['discount'] = x['discount']*100

<ipython-input-29-19f8c0a34c58>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['discount'] = x['discount']*100


In [30]:
x = sm.add_constant(x)

In [31]:
logit_model = sm.Logit(y, x)

result = logit_model.fit(disp=False)

print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                15508
Model:                          Logit   Df Residuals:                    15504
Method:                           MLE   Df Model:                            3
Date:                Wed, 03 Dec 2025   Pseudo R-squ.:                0.007033
Time:                        17:24:47   Log-Likelihood:                -7446.1
converged:                       True   LL-Null:                       -7498.8
Covariance Type:            nonrobust   LLR p-value:                 1.034e-22
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6909      0.034    -49.413      0.000      -1.758      -1.624
discount       0.0054      0.001      6.209      0.000       0.004       0.007
likes       3.286e-06   4.57e-07      7.197      0.0

In [32]:
data['prob'] = result.predict(x)
data['pred'] = (data['prob'] >= 0.5).astype(int)

In [33]:
from sklearn.metrics import accuracy_score

y_true = y
y_pred = data['pred']

acc = accuracy_score(y_true, y_pred)
print("Accuracy", acc)

Accuracy 0.8117745679649213


In [ ]:
# AI model

In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [72]:
sales_dummy = []
for i in range(len(data)):
    if data['sales'][i]==0:
        sales_dummy.append(0)
    else:
        sales_dummy.append(1)
data['sales_dummy'] = sales_dummy

In [73]:
x = data[['likes', 'discount', 'shippirce']]
y = data['sales_dummy']

In [74]:
x = x.values
y = y.values

In [75]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [76]:
model = Sequential([
    Dense(32, activation='sigmoid', input_shape=(x.shape[1],)),  
    Dense(16, activation='relu'),                      
    Dense(1, activation='sigmoid')                    
])


In [77]:
model.compile(optimizer=Adam(learning_rate=0.1),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x, y, epochs=20, verbose=1)

loss, acc = model.evaluate(x, y)
print("Accuracy:", acc)

Epoch 1/20
485/485 [==============================] - 3s 5ms/step - loss: 0.4880 - accuracy: 0.8116
Epoch 2/20
485/485 [==============================] - 3s 5ms/step - loss: 0.4848 - accuracy: 0.8118
Epoch 3/20
485/485 [==============================] - 3s 6ms/step - loss: 0.4849 - accuracy: 0.8118
Epoch 4/20
485/485 [==============================] - 4s 8ms/step - loss: 0.4846 - accuracy: 0.8118
Epoch 5/20
485/485 [==============================] - 3s 6ms/step - loss: 0.4851 - accuracy: 0.8118
Epoch 6/20
485/485 [==============================] - 3s 6ms/step - loss: 0.4851 - accuracy: 0.8118
Epoch 7/20
485/485 [==============================] - 3s 6ms/step - loss: 0.4857 - accuracy: 0.8118
Epoch 8/20
485/485 [==============================] - 3s 6ms/step - loss: 0.4849 - accuracy: 0.8118
Epoch 9/20
485/485 [==============================] - 3s 5ms/step - loss: 0.4851 - accuracy: 0.8118
Epoch 10/20
485/485 [==============================] - 3s 6ms/step - loss: 0.4853 - accuracy: 0.8118